# Imports

## Libraries and Packages

In [4]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

## Function

In [5]:
from os import listdir

def Locate_Files(path_to_dir, suffix=(".csv")):
    """
    path_to_dir = path to files
    suffix = file extension (must be tuple if there's more than one file extension needed)
    """
    
    #Fetch all Files
    filenames = listdir(path_to_dir)
    
    #Filter all Files
    filenames = [filename for filename in filenames if filename.endswith(suffix)]
    
    #Add the full Path
    filenames = [path_to_dir + v for v in filenames]
    
    return filenames

## Data

### Case History

In [113]:
HISTORY_DIR = "../Data/Case History/"
TO_PROCESS = Locate_Files(HISTORY_DIR, (".csv"))

DF_HISTORY = []
for file in TO_PROCESS:
    a = pd.read_csv(file)
    DF_HISTORY.append(a)


DF_HISTORY = pd.concat(DF_HISTORY, axis=0)

### Survey

In [7]:
SURVEY_DIR = "../Data/Survey/"
TO_PROCESS = Locate_Files(SURVEY_DIR, (".csv"))

DF_SURVEY = []
for file in TO_PROCESS:
    a = pd.read_csv(file)
    DF_SURVEY.append(a)


DF_SURVEY = pd.concat(DF_SURVEY, axis=0)


### Case Details

In [83]:
CASE_DIR = "../Data/Case Details/"
TO_PROCESS = Locate_Files(CASE_DIR, (".csv"))

DF_CASE = []
for file in TO_PROCESS:
    a = pd.read_csv(file)
    DF_CASE.append(a)


DF_CASE = pd.concat(DF_CASE, axis=0)
#DF_CASE = DF_CASE[["Case Number", "Case Origin"]]

In [88]:
DF_CASE[DF_CASE["Status"].isin(["Closed","Resolved"])]

,Account Owner,Case Number,Date/Time Opened,Date/Time Closed,First Queue Assigned,Last Queue Assigned,Last Macro Applied,Submitted Category,Submitted Subcategory,User Value Risk,Description,Monorail Country Code,L1 Workflow,L2 Workflow,L3 Workflow,Team,Sub Team,Status,Case Origin
0,Mulesoft Cases Integration User,7667167,10/1/2021 12:00 AM,10/11/2021 4:03 PM,T1 Support Queue - Wallet,T1 Support Queue - Wallet,Coinbase Wallet - Adjusting Miner Fees 101,Send_Receive_Crypto,Outgoing_transactions_from_my_Coinbase_Wallet,3.0,Coinbase wallet will not let me withdraw or tr...,US,Payments - Crypto,Crypto Ops,Wallet,BPO Ops,Other,Closed,Web
1,NaN,7673502,10/1/2021 12:15 PM,NaN,CS - Exchange,CS - Exchange,NaN,NaN,NaN,NaN,My account still shows up with a $35k limit. ...,NaN,NaN,NaN,NaN,Client Services,Exchange,Resolved,ClientServices@
2,Mulesoft Cases Integration User,7680635,10/2/2021 6:21 AM,10/10/2021 4:08 PM,T1 Support Queue - Wallet,T2 - SME Wallet Queue,[Archived] General > Free Form Template #2 - E...,Unable_to_Use_Account,My_account_is_restricted,2.0,Please fix this,PH,Payments - Crypto,Crypto Ops,Wallet,Support - Wallet,Other,Closed,Web
3,Mulesoft Cases Integration User,7686892,10/2/2021 10:08 PM,10/17/2021 4:22 PM,T1 - Compliance,T1 - Access,Account Recovery - FM Troubleshooting,I_need_help_verifying_my_identity,I_need_to_verify_my_ID,2.0,I Do not have the same phone number,US,Account Ops,Account Access,Troubleshooting,Account Ops,Access,Closed,Web
4,NaN,7693184,10/3/2021 4:08 PM,NaN,NaN,NaN,Silent Resolve,NaN,NaN,NaN,Call From: +19734272708 \nDialing Coinbase Sup...,NaN,General,Other,NaN,Other,Other,Resolved,Phone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68605,Mulesoft Cases Integration User,8464249,11/10/2021 11:58 PM,11/24/2021 5:35 PM,T1 DE - General,T2 Support Queue - SME Fiat - SEPA,de_DE - SEPA - Unmatched SEPA Deposit - Invali...,product_suggestion,General,4.0,ich habe eine 2 Überweisung gemacht IN WERT V...,DE,Payment Ops,SEPA,SEPA Issues (CAT),Support - Payments,Payments,Closed,Web
68606,Mulesoft Cases Integration User,8464251,11/10/2021 11:59 PM,12/2/2021 5:33 PM,T1 Support Queue - Payments,T2 - Risk,Fraud > Clawback - No Credit / Refunds,Buy_Sell_Convert_Crypto,Issues_with_Buying,3.0,"Hi,\nI bought $1000 bitcoin today at 11:43 PST...",US,Identity Ops,Account Risk,Clawback Process,Account Ops,Risk,Closed,Web
68607,Mulesoft Cases Integration User,8464252,11/10/2021 11:59 PM,11/16/2021 4:14 PM,T1 Support Queue - Payments,T1 Support Queue - Payments,General - More Info Needed,Buy_Sell_Convert_Crypto,Issues_with_Buying,3.0,I opened a order at 16:18 on 11/10/2021 for 10...,US,Retail Ops,Other,Multiple Issues,BPO Ops,Payments,Closed,Web
68608,Mulesoft Cases Integration User,8464253,11/10/2021 11:59 PM,11/16/2021 5:39 PM,T1 - Compliance,T1 - Compliance,IDV - Verified,Account_Info_and_IDV,ID_Verification,3.0,Coinbase is asking my ID to be from Canada. Bu...,CA,Policy Ops,Retail Onboarding,IDV (Jumio),Policy Ops,Compliance,Closed,Web


# Cleaning

## For Survey

In [9]:
DF_SURVEY.rename(columns={"Case: Case Number": "Case Number"}, inplace=True)

In [10]:
DF_SURVEY["Resolution"] = \
DF_SURVEY["Was your issue resolved?"].str.lower().str.contains("no").apply(lambda x: 0 if x==True else 1)

## For Case History

In [114]:
DF_HISTORY["Case Date/Time Last Modified"] = pd.to_datetime(DF_HISTORY["Case Date/Time Last Modified"])
DF_HISTORY["Edit Date"] = pd.to_datetime(DF_HISTORY["Edit Date"])

In [115]:
DF_HISTORY = DF_HISTORY.sort_values(by=["Case Number", "Edit Date"])

In [116]:
DF_HISTORY["Edit Week"] = DF_HISTORY["Edit Date"].dt.to_period("W").apply(lambda r: r.start_time)

In [117]:
DF_HISTORY = DF_HISTORY.merge(DF_CASE, on="Case Number")

In [118]:
DF_HISTORY["Case Number"].nunique()

1134431

In [119]:
DF_HISTORY = DF_HISTORY[DF_HISTORY["Case Origin"].isin(["Live Message","Phone","Web"])]

In [120]:
QUERY = \
DF_HISTORY[DF_HISTORY["Field / Event"] == "L1 Workflow"].groupby(["New Value","Case Origin"])["Case Number"].nunique().unstack()[["Live Message","Phone","Web"]].fillna(0).astype(int)

QUERY["Total"] = QUERY.sum(axis=1)
QUERY["% Total"] = QUERY["Total"].div(QUERY["Total"].sum()).round(4)
QUERY = QUERY.sort_values(by="Total",ascending=False)

### General Volume

In [121]:
QUERY

Case Origin,Live Message,Phone,Web,Total,% Total
New Value,,,,,
Identity Ops,25087,101124,306099,432310,0.3669
Ecosystem Ops,11872,17823,152917,182612,0.1550
Policy Ops,3716,7359,156639,167714,0.1423
Payment Ops,8299,14974,131161,154434,0.1311
Retail Ops,3707,7212,98281,109200,0.0927
Institutional Ops,2545,1958,46606,51109,0.0434
General,2348,22694,5868,30910,0.0262
Trading,482,2692,13716,16890,0.0143
Banking Ops,871,5063,10379,16313,0.0138


In [125]:
QUERY["Total"].sum()

1178400

In [124]:
{"Identity Ops": "Account Ops", "Payments - Crypto": "Ecosystem Ops", "Payments - Fiat": "Retail Ops"}

{'Identity Ops': 'Account Ops',
 'Payments - Crypto': 'Ecosystem Ops',
 'Payments - Fiat': 'Retail Ops'}

In [126]:
QUERY = \
DF_HISTORY[(DF_HISTORY["Field / Event"] == "L1 Workflow")].groupby(["New Value","Case Origin"])["Case Number"].nunique().unstack()[["Live Message","Phone","Web"]].fillna(0).astype(int)

QUERY
#QUERY["Total"] = QUERY.sum(axis=1)
#QUERY["% Total"] = QUERY["Total"].div(QUERY["Total"].sum()).round(4)
#QUERY = QUERY.sort_values(by="Total",ascending=False)

Case Origin,Live Message,Phone,Web
New Value,,,
Account Ops,0,1484,7038
Banking Ops,871,5063,10379
Client Services,0,0,540
Community Ops,25,50,39
Ecosystem Ops,11872,17823,152917
General,2348,22694,5868
Identity Ops,25087,101124,306099
Institutional Ops,2545,1958,46606
Payment Ops,8299,14974,131161


In [153]:
DF_HISTORY_STATUS = DF_HISTORY[DF_HISTORY["Field / Event"] == "Status"].reset_index(drop=True).copy()

In [154]:
DF_HISTORY_STATUS.groupby(DF_HISTORY_STATUS["Edit Date"].dt.month_name())["Case Number"].nunique()

Edit Date
December    420326
November    530126
October     351745
Name: Case Number, dtype: int64

In [155]:
DF_HISTORY_STATUS.groupby(["Edit Week"])["Case Number"].nunique()

Edit Week
2021-09-27     15518
2021-10-04     84099
2021-10-11    109639
2021-10-18    119115
2021-10-25    179561
2021-11-01    218369
2021-11-08    194884
2021-11-15    172576
2021-11-22    160453
2021-11-29    148637
2021-12-06    141006
2021-12-13    116028
2021-12-20    125500
2021-12-27     99669
Name: Case Number, dtype: int64

In [177]:
DF_HISTORY_STATUS_PSEUDO = DF_HISTORY_STATUS[~(DF_HISTORY_STATUS["Case Number"].duplicated(keep="last"))].reset_index(drop=True).copy()

In [178]:
DF_HISTORY_STATUS_PSEUDO.groupby(["Edit Week"])["Case Number"].nunique().sum()

1094900

In [179]:
DF_HISTORY_STATUS_PSEUDO.groupby(["Edit Week","Case Origin"])["Case Number"].nunique().unstack()[["Web", "Phone", "Live Message"]].fillna(0).astype(int)

Case Origin,Web,Phone,Live Message
Edit Week,,,
2021-09-27,1359,2637,0
2021-10-04,18338,12045,0
2021-10-11,50221,10915,0
2021-10-18,51535,13114,0
2021-10-25,61710,11616,12
2021-11-01,109471,5870,30
2021-11-08,107224,5808,22
2021-11-15,86545,7924,211
2021-11-22,77184,14014,51


In [144]:
DF_HISTORY_STATUS_PSEUDO.groupby(["Edit Week","Case Origin"])["Case Number"].nunique().unstack()[["Web", "Phone", "Live Message"]].fillna(0).astype(int).sum().sum()

1094900

In [145]:
DF_HISTORY_STATUS_PSEUDO.groupby(["Edit Week","Case Origin"])["Case Number"].nunique().unstack()[["Web", "Phone", "Live Message"]].fillna(0).astype(int)

Case Origin,Web,Phone,Live Message
Edit Week,,,
2021-09-27,1359,2637,0
2021-10-04,18338,12045,0
2021-10-11,50221,10915,0
2021-10-18,51535,13114,0
2021-10-25,61710,11616,12
2021-11-01,109471,5870,30
2021-11-08,107224,5808,22
2021-11-15,86545,7924,211
2021-11-22,77184,14014,51


In [146]:
DF_HISTORY_STATUS_PSEUDO.groupby(["Edit Week","Case Origin"])["Case Number"].nunique().unstack()[["Web", "Phone", "Live Message"]].fillna(0).astype(int).sum().sum()

1094900

In [191]:
DF_CASE.head(2)

,Account Owner,Case Number,Date/Time Opened,Date/Time Closed,First Queue Assigned,Last Queue Assigned,Last Macro Applied,Submitted Category,Submitted Subcategory,User Value Risk,Description,Monorail Country Code,L1 Workflow,L2 Workflow,L3 Workflow,Team,Sub Team,Status,Case Origin
0,Mulesoft Cases Integration User,7667167,10/1/2021 12:00 AM,10/11/2021 4:03 PM,T1 Support Queue - Wallet,T1 Support Queue - Wallet,Coinbase Wallet - Adjusting Miner Fees 101,Send_Receive_Crypto,Outgoing_transactions_from_my_Coinbase_Wallet,3.0,Coinbase wallet will not let me withdraw or tr...,US,Payments - Crypto,Crypto Ops,Wallet,BPO Ops,Other,Closed,Web
1,NaN,7673502,10/1/2021 12:15 PM,NaN,CS - Exchange,CS - Exchange,NaN,NaN,NaN,NaN,My account still shows up with a $35k limit. ...,NaN,NaN,NaN,NaN,Client Services,Exchange,Resolved,ClientServices@


In [193]:
DF_HISTORY_STATUS_PSEUDO = \
DF_HISTORY_STATUS_PSEUDO.merge(DF_CASE[["Case Number", "Status"]], on="Case Number", how="left")

In [200]:
DF_HISTORY_STATUS_PSEUDO["Status_x"]

0            Closed
1            Closed
2            Closed
3            Closed
4            Closed
             ...   
1094895      Closed
1094896      Closed
1094897      Closed
1094898      Closed
1094899    Resolved
Name: Status_x, Length: 1094900, dtype: object

In [202]:
DF_HISTORY_STATUS_PSEUDO["Status_x"].value_counts()

Closed                920732
Resolved              134829
Open - In Progress     33541
Pending Customer        2898
Escalated               1437
Pending Internal         804
Transferred              362
Customer Responded       295
New                        2
Name: Status_x, dtype: int64

In [203]:
920732 + 134829

1055561

## Get All Reopened Cases

In [156]:
CASE_NUMBERS = []
L1_WORKFLOWS = []
L2_WORKFLOWS = []
L3_WORKFLOWS = []
MACRO = []
DATE_EDITED = []
DATE_EDITED_PRIOR = []

Case_Num = 0
Get_L1_Workflow = "abcdef"
Get_L2_Workflow = "abcdef"
Get_L3_Workflow = "abcdef"
Get_Macro = "abcdef"
Get_Date_Prior = "abcdef"

for case,event,old_val,new_val,date_edit in zip(DF_HISTORY["Case Number"],
                                                DF_HISTORY["Field / Event"],
                                                DF_HISTORY["Old Value"],
                                                DF_HISTORY["New Value"],
                                                DF_HISTORY["Edit Date"]):
    
    if Case_Num != case:
        Case_Num = case
        
    if event == "Last Macro Applied":
        Get_Macro = new_val
        
    if event == "L1 Workflow":
        Get_L1_Workflow = new_val
        
    if event == "L2 Workflow":
        Get_L2_Workflow = new_val
        
    if event == "L3 Workflow":
        Get_L3_Workflow = new_val
        
    if event == "L3 Workflow":
        Get_L3_Workflow = new_val
        
    if event == "Status":
        if (old_val == "Resolved") & (new_val == "Customer Responded"):
            
            CASE_NUMBERS.append(Case_Num)
            DATE_EDITED.append(date_edit)
            DATE_EDITED_PRIOR.append(Get_Date_Prior)
            MACRO.append(Get_Macro)
            L1_WORKFLOWS.append(Get_L1_Workflow)
            L2_WORKFLOWS.append(Get_L2_Workflow)
            L3_WORKFLOWS.append(Get_L3_Workflow)
            
            Case_Num = 0
            Get_L1_Workflow = "abcdef"
            Get_L2_Workflow = "abcdef"
            Get_L3_Workflow = "abcdef"
            Get_Macro = "abcdef"
            Get_Date_Prior = "abcdef"
            
        else:
            Get_Date_Prior = date_edit


In [221]:
DF_CONSOLIDATED = \
pd.concat([pd.Series(CASE_NUMBERS).rename("Case Number"),
           pd.Series(DATE_EDITED_PRIOR).rename("Status Change Prior"),
           pd.Series(DATE_EDITED).rename("Date/Time Reopened"),
           pd.Series(L1_WORKFLOWS).rename("L1 Workflow"),
           pd.Series(L2_WORKFLOWS).rename("L2 Workflow"),
           pd.Series(L3_WORKFLOWS).rename("L3 Workflow"),
           pd.Series(MACRO).rename("Macro Applied")
          ], axis=1)

In [222]:
for col in DF_CONSOLIDATED.columns:
    try:
        DF_CONSOLIDATED[col] = DF_CONSOLIDATED[col].replace("abcdef",np.nan)
    except:
        print(col)

In [223]:
DF_CONSOLIDATED.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 427021 entries, 0 to 427020
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Case Number          427021 non-null  int64         
 1   Status Change Prior  424207 non-null  datetime64[ns]
 2   Date/Time Reopened   427021 non-null  datetime64[ns]
 3   L1 Workflow          271866 non-null  object        
 4   L2 Workflow          280381 non-null  object        
 5   L3 Workflow          296017 non-null  object        
 6   Macro Applied        397733 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 22.8+ MB


In [224]:
DF_CONSOLIDATED["Case Number"].nunique()

242328

In [225]:
DF_CONSOLIDATED = DF_CONSOLIDATED[~(DF_CONSOLIDATED["Status Change Prior"].isnull())].reset_index(drop=True)

In [226]:
DF_CONSOLIDATED["Interval (Days)"] = \
(DF_CONSOLIDATED["Date/Time Reopened"] - DF_CONSOLIDATED["Status Change Prior"]).dt.days

In [227]:
DF_CONSOLIDATED = DF_CONSOLIDATED[~(DF_CONSOLIDATED["Interval (Days)"] < 0)].reset_index(drop=True)

In [228]:
TO_CHECK = DF_CONSOLIDATED["Interval (Days)"]

CONDITIONS = [TO_CHECK <= 7]

CHOICES = ["Within 7 Days"]

DF_CONSOLIDATED["FCR"] = np.select(CONDITIONS, CHOICES, "Outside 5 Days")

In [229]:
DF_CONSOLIDATED["Workweek"] = DF_CONSOLIDATED["Date/Time Reopened"].dt.to_period("W").apply(lambda r: r.start_time)

In [230]:
DF_CONSOLIDATED = DF_CONSOLIDATED.sort_values(by="Workweek")
DF_CONSOLIDATED.head()

,Case Number,Status Change Prior,Date/Time Reopened,L1 Workflow,L2 Workflow,L3 Workflow,Macro Applied,Interval (Days),FCR,Workweek
0,7667167,2021-10-02 05:39:00,2021-10-03 18:29:00,Payments - Crypto,Crypto Ops,Wallet,Coinbase Wallet - Miner Fee Error,1,Within 7 Days,2021-09-27
4042,7681682,2021-10-02 09:03:00,2021-10-02 12:07:00,Client Services,Pro,Address Book & Whitelisting,General > Bug > Basic Troubleshooting - Clear ...,0,Within 7 Days,2021-09-27
4039,7681664,2021-10-02 11:42:00,2021-10-02 13:16:00,Payments - Fiat,ACH,Cancelled Buy / Deposit,"Cancelled Purchase - Manual, No Flag",0,Within 7 Days,2021-09-27
4038,7681656,2021-10-02 10:45:00,2021-10-02 11:31:00,Payments - Fiat,ACH,Veritification ACH,ACH - Bank Account Reinstated,0,Within 7 Days,2021-09-27
4034,7681650,2021-10-03 02:49:00,2021-10-03 06:51:00,Account Ops,Account Safety,Unauthorized Crypto Buy/Sell/Send,Self Recovery Already Processing,0,Within 7 Days,2021-09-27


In [231]:
DF_CONSOLIDATED = DF_CONSOLIDATED.merge(DF_CASE,on="Case Number")

In [232]:
DF_CONSOLIDATED = \
DF_CONSOLIDATED[DF_CONSOLIDATED["Case Origin"].isin(["Web", "Phone", "Live Message"])].reset_index(drop=True)

In [233]:
DF_CONSOLIDATED["Interval (Days)"].isnull().sum()

0

In [234]:
TO_CHECK = DF_CONSOLIDATED["Interval (Days)"]

CONDITIONS = [TO_CHECK == 0,
              (TO_CHECK >= 1) & (TO_CHECK <= 2),
              (TO_CHECK >= 3) & (TO_CHECK <= 4),
              (TO_CHECK >= 5) & (TO_CHECK <= 7)
             ]

CHOICES = ["Same Day",
           "1-2 Day",
           "3-4 Days",
           "5-7 Days"
          ]

DF_CONSOLIDATED["Duration_Reopened"] = np.select(CONDITIONS, CHOICES, "> 7 Days")

In [235]:
DF_CONSOLIDATED.head()

,Case Number,Status Change Prior,Date/Time Reopened,L1 Workflow_x,L2 Workflow_x,L3 Workflow_x,Macro Applied,Interval (Days),FCR,Workweek,...,Description,Monorail Country Code,L1 Workflow_y,L2 Workflow_y,L3 Workflow_y,Team,Sub Team,Status,Case Origin,Duration_Reopened
0,7667167,2021-10-02 05:39:00,2021-10-03 18:29:00,Payments - Crypto,Crypto Ops,Wallet,Coinbase Wallet - Miner Fee Error,1,Within 7 Days,2021-09-27,...,Coinbase wallet will not let me withdraw or tr...,US,Payments - Crypto,Crypto Ops,Wallet,BPO Ops,Other,Closed,Web,1-2 Day
1,7681682,2021-10-02 09:03:00,2021-10-02 12:07:00,Client Services,Pro,Address Book & Whitelisting,General > Bug > Basic Troubleshooting - Clear ...,0,Within 7 Days,2021-09-27,...,i'm not able to create an address in my addres...,US,Client Services,Pro,Deposits and Withdrawals (Crypto),Client Services,Pro,Closed,Web,Same Day
2,7681682,2021-10-03 00:13:00,2021-10-03 06:55:00,NaN,NaN,NaN,Pages - Wallet Addresses,0,Within 7 Days,2021-09-27,...,i'm not able to create an address in my addres...,US,Client Services,Pro,Deposits and Withdrawals (Crypto),Client Services,Pro,Closed,Web,Same Day
3,7681682,2021-10-04 09:33:00,2021-10-04 11:54:00,NaN,NaN,NaN,Pro - Funds on Hold for Open Order,0,Within 7 Days,2021-10-04,...,i'm not able to create an address in my addres...,US,Client Services,Pro,Deposits and Withdrawals (Crypto),Client Services,Pro,Closed,Web,Same Day
4,7681682,2021-10-04 08:28:00,2021-10-04 08:44:00,NaN,NaN,Deposits and Withdrawals (Crypto),General > Free Form Template #1 - General,0,Within 7 Days,2021-10-04,...,i'm not able to create an address in my addres...,US,Client Services,Pro,Deposits and Withdrawals (Crypto),Client Services,Pro,Closed,Web,Same Day


In [236]:
DF_CONSOLIDATED = DF_CONSOLIDATED.merge(DF_SURVEY[["Case Number","Resolution"]], on="Case Number", how="left")

In [237]:
DF_CONSOLIDATED["Surveyed"] = DF_CONSOLIDATED["Resolution"].isnull().replace({True: 1, False: 0})

In [238]:
GET_SOMETHING = \
DF_CONSOLIDATED[~(DF_CONSOLIDATED["Case Number"].duplicated(keep="last"))]

In [239]:
pd.concat([GET_SOMETHING.groupby(["Duration_Reopened"])["Resolution"].mean(),
           GET_SOMETHING.groupby(["Duration_Reopened"])["Surveyed"].mean(),
           GET_SOMETHING.groupby(["Duration_Reopened"])["Surveyed"].count()
          ],axis=1).reindex(["Same Day","1-2 Day","3-4 Days", "5-7 Days", "> 7 Days"])


,Resolution,Surveyed,Surveyed
Duration_Reopened,,,
Same Day,0.207676,0.515486,203862
1-2 Day,0.090367,0.461075,22094
3-4 Days,0.083436,0.481222,9426
5-7 Days,0.087294,0.509966,4465
> 7 Days,0.087600,0.598447,2189


# EDA

In [240]:
DF_CONSOLIDATED["Case Origin"].value_counts()

Web             395366
Phone            24442
Live Message      4210
Name: Case Origin, dtype: int64

In [241]:
DF_CONSOLIDATED

,Case Number,Status Change Prior,Date/Time Reopened,L1 Workflow_x,L2 Workflow_x,L3 Workflow_x,Macro Applied,Interval (Days),FCR,Workweek,...,L1 Workflow_y,L2 Workflow_y,L3 Workflow_y,Team,Sub Team,Status,Case Origin,Duration_Reopened,Resolution,Surveyed
0,7667167,2021-10-02 05:39:00,2021-10-03 18:29:00,Payments - Crypto,Crypto Ops,Wallet,Coinbase Wallet - Miner Fee Error,1,Within 7 Days,2021-09-27,...,Payments - Crypto,Crypto Ops,Wallet,BPO Ops,Other,Closed,Web,1-2 Day,0.0,0
1,7681682,2021-10-02 09:03:00,2021-10-02 12:07:00,Client Services,Pro,Address Book & Whitelisting,General > Bug > Basic Troubleshooting - Clear ...,0,Within 7 Days,2021-09-27,...,Client Services,Pro,Deposits and Withdrawals (Crypto),Client Services,Pro,Closed,Web,Same Day,0.0,0
2,7681682,2021-10-03 00:13:00,2021-10-03 06:55:00,NaN,NaN,NaN,Pages - Wallet Addresses,0,Within 7 Days,2021-09-27,...,Client Services,Pro,Deposits and Withdrawals (Crypto),Client Services,Pro,Closed,Web,Same Day,0.0,0
3,7681682,2021-10-04 09:33:00,2021-10-04 11:54:00,NaN,NaN,NaN,Pro - Funds on Hold for Open Order,0,Within 7 Days,2021-10-04,...,Client Services,Pro,Deposits and Withdrawals (Crypto),Client Services,Pro,Closed,Web,Same Day,0.0,0
4,7681682,2021-10-04 08:28:00,2021-10-04 08:44:00,NaN,NaN,Deposits and Withdrawals (Crypto),General > Free Form Template #1 - General,0,Within 7 Days,2021-10-04,...,Client Services,Pro,Deposits and Withdrawals (Crypto),Client Services,Pro,Closed,Web,Same Day,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424013,9261058,2021-12-30 06:22:00,2021-12-30 07:05:00,Payment Ops,SEPA,SEPA Issues (CAT),SEPA - Add Cash Successful,0,Within 7 Days,2021-12-27,...,Payment Ops,SEPA,SEPA Issues (CAT),Client Services,Pro,Closed,Web,Same Day,NaN,1
424014,9261050,2021-12-27 05:03:00,2021-12-27 05:10:00,Payment Ops,PayPal,PayPal Withdraw / Sell Issue,PayPal - Unclaimed Funds,0,Within 7 Days,2021-12-27,...,Payment Ops,PayPal,PayPal Withdraw / Sell Issue,Support - Payments,Payments,Closed,Web,Same Day,0.0,0
424015,9261027,2021-12-28 08:49:00,2021-12-28 09:05:00,NaN,NaN,NaN,SEPA - How to Add Cash EUR with a Bank Transfer,0,Within 7 Days,2021-12-27,...,Payment Ops,SEPA,SEPA Issues (CAT),Support - Payments,Payments,Closed,Web,Same Day,NaN,1
424016,9261027,2021-12-28 08:01:00,2021-12-28 08:21:00,Payment Ops,SEPA,SEPA Issues (CAT),Payments - Deposit Successful,0,Within 7 Days,2021-12-27,...,Payment Ops,SEPA,SEPA Issues (CAT),Support - Payments,Payments,Closed,Web,Same Day,NaN,1


In [242]:
DF_CONSOLIDATED_PSEUDO = DF_CONSOLIDATED[~(DF_CONSOLIDATED["Case Number"].duplicated(keep="last"))]

In [245]:
DF_CONSOLIDATED_PSEUDO.groupby(["L1 Workflow_x"])["Case Number"].nunique().reset_index()

,L1 Workflow_x,Case Number
0,Account Ops,1565
1,Banking Ops,1577
2,Client Services,128
3,Community Ops,2
4,Ecosystem Ops,33941
5,General,1039
6,Identity Ops,60183
7,Institutional Ops,8195
8,Payment Ops,26600
9,Payments - Crypto,1014


In [248]:
DF_CONSOLIDATED_PSEUDO.groupby(["L1 Workflow_x", "Case Origin"])["Case Number"].nunique().unstack().fillna(0).astype(int)

Case Origin,Live Message,Phone,Web
L1 Workflow_x,,,
Account Ops,0,78,1487
Banking Ops,16,179,1382
Client Services,0,0,128
Community Ops,0,0,2
Ecosystem Ops,413,1475,32053
General,5,298,736
Identity Ops,594,6686,52903
Institutional Ops,323,193,7679
Payment Ops,221,1053,25326


In [250]:
DF_CONSOLIDATED_PSEUDO["Case Origin"].value_counts(normalize=True)

Web             0.929531
Phone           0.059859
Live Message    0.010610
Name: Case Origin, dtype: float64

In [244]:
DF_CONSOLIDATED_PSEUDO.head()

,Case Number,Status Change Prior,Date/Time Reopened,L1 Workflow_x,L2 Workflow_x,L3 Workflow_x,Macro Applied,Interval (Days),FCR,Workweek,...,L1 Workflow_y,L2 Workflow_y,L3 Workflow_y,Team,Sub Team,Status,Case Origin,Duration_Reopened,Resolution,Surveyed
0,7667167,2021-10-02 05:39:00,2021-10-03 18:29:00,Payments - Crypto,Crypto Ops,Wallet,Coinbase Wallet - Miner Fee Error,1,Within 7 Days,2021-09-27,...,Payments - Crypto,Crypto Ops,Wallet,BPO Ops,Other,Closed,Web,1-2 Day,0.0,0
4,7681682,2021-10-04 08:28:00,2021-10-04 08:44:00,NaN,NaN,Deposits and Withdrawals (Crypto),General > Free Form Template #1 - General,0,Within 7 Days,2021-10-04,...,Client Services,Pro,Deposits and Withdrawals (Crypto),Client Services,Pro,Closed,Web,Same Day,0.0,0
5,7681664,2021-10-02 11:42:00,2021-10-02 13:16:00,Payments - Fiat,ACH,Cancelled Buy / Deposit,"Cancelled Purchase - Manual, No Flag",0,Within 7 Days,2021-09-27,...,Payments - Fiat,ACH,Cancelled Buy / Deposit,Account Ops,Risk,Closed,Web,Same Day,0.0,0
6,7681656,2021-10-02 10:45:00,2021-10-02 11:31:00,Payments - Fiat,ACH,Veritification ACH,ACH - Bank Account Reinstated,0,Within 7 Days,2021-09-27,...,Payments - Fiat,ACH,Veritification ACH,Support - Payments,Payments,Closed,Web,Same Day,0.0,0
8,7681650,2021-10-03 08:37:00,2021-10-05 08:45:00,NaN,NaN,NaN,NaN,2,Within 7 Days,2021-10-04,...,Account Ops,Account Safety,Unauthorized Crypto Buy/Sell/Send,BPO Ops,BPO General,Closed,Web,1-2 Day,0.0,0


In [220]:
DF_CONSOLIDATED_PSEUDO.groupby(["Workweek","L1 Workflow"])["Case Number"].nunique().unstack().fillna(0).astype(int)

KeyError: 'L1 Workflow'

In [ ]:
DF_CONSOLIDATED_PSEUDO["Case Origin"].value_counts().sum()

In [213]:
DF_CONSOLIDATED.groupby(["Workweek","L1 Workflow"])["Case Number"].nunique().unstack()[["Web", "Phone", "Live Message"]].fillna(0).astype(int)

KeyError: 'L1 Workflow'

In [184]:
DF_CONSOLIDATED["Case Number"].nunique()

242036

In [76]:
242038/1178400

0.20539545145960625

In [204]:
DF_CONSOLIDATED.to_excel("All_Reopened Cases.xlsx", index=False)

In [205]:
DF_CONSOLIDATED[~(DF_CONSOLIDATED["Case Number"].duplicated(keep="last"))].to_excel("All_Reopened Cases_Unique.xlsx", index=False)

In [180]:
QUERY_WEB = \
DF_CONSOLIDATED[DF_CONSOLIDATED["Case Origin"] == "Web"]

QUERY_PHONE = \
DF_CONSOLIDATED[DF_CONSOLIDATED["Case Origin"] == "Phone"]

QUERY_LM = \
DF_CONSOLIDATED[DF_CONSOLIDATED["Case Origin"] == "Live Message"]

QUERY_WEB = QUERY_WEB.merge(DF_SURVEY[["Case Number", "Resolution"]], on="Case Number", how="left")
QUERY_PHONE = QUERY_PHONE.merge(DF_SURVEY[["Case Number", "Resolution"]], on="Case Number", how="left")
QUERY_LM = QUERY_LM.merge(DF_SURVEY[["Case Number", "Resolution"]], on="Case Number", how="left")

In [62]:
QUERY_PHONE.groupby(["Case Number","L1 Workflow"])["Status Change Prior"].count().unstack().mean()

L1 Workflow
Account Ops          1.000000
Banking Ops          1.004464
Ecosystem Ops        1.022037
General              1.002646
Identity Ops         1.011724
Institutional Ops    1.015094
Payment Ops          1.011445
Payments - Crypto    1.000000
Payments - Fiat      1.000000
Policy Ops           1.020571
Retail Ops           1.013221
Trading              1.000000
dtype: float64

In [63]:
QUERY_WEB["L1 Workflow"].value_counts().reset_index()

,index,L1 Workflow
0,Identity Ops,76727
1,Ecosystem Ops,49436
2,Policy Ops,37593
3,Payment Ops,35373
4,Retail Ops,30984
5,Institutional Ops,11395
6,Trading,3943
7,Banking Ops,2113
8,Account Ops,2030
9,Payments - Crypto,1493


In [64]:
QUERY_PHONE["L1 Workflow"].value_counts().reset_index()

,index,L1 Workflow
0,Identity Ops,9147
1,Ecosystem Ops,2087
2,Payment Ops,1414
3,Policy Ops,893
4,Retail Ops,843
5,General,379
6,Institutional Ops,269
7,Trading,244
8,Banking Ops,225
9,Account Ops,118


In [65]:
QUERY_LM["L1 Workflow"].value_counts().reset_index()

,index,L1 Workflow
0,Identity Ops,765
1,Ecosystem Ops,517
2,Policy Ops,431
3,Institutional Ops,418
4,Retail Ops,298
5,Payment Ops,282
6,Banking Ops,20
7,Trading,19
8,General,5


## Phone Reopened Cases

In [87]:
QUERY_PHONE.groupby(["Workweek"])["Case Number"].nunique().rename("Count").reset_index()

,Workweek,Count
0,2021-09-27,29
1,2021-10-04,263
2,2021-10-11,283
3,2021-10-18,318
4,2021-10-25,522
5,2021-11-01,603
6,2021-11-08,647
7,2021-11-15,1067
8,2021-11-22,1889
9,2021-11-29,2130


### Phone: Identity Ops

In [127]:
QUERY = QUERY_PHONE[QUERY_PHONE["L1 Workflow"] == "Identity Ops"].reset_index(drop=True)

In [128]:
pd.concat([QUERY["L2 Workflow"].value_counts().rename("Count"),
           QUERY["L2 Workflow"].value_counts(normalize=True).round(4).rename("% Count"),
           QUERY.groupby(["L2 Workflow"])["Resolution"].mean().round(4)
          ],axis=1)

,Count,% Count,Resolution
Account Safety,4547,0.4973,0.1248
Account Risk,2532,0.2769,0.1389
Account Access,1900,0.2078,0.1786
Other,164,0.0179,0.1731
Compliance Policy,1,0.0001,0.0000


In [129]:
pd.concat([QUERY["L3 Workflow"].value_counts().rename("Count"),
           QUERY["L3 Workflow"].value_counts(normalize=True).round(4).rename("% Count"),
           QUERY.groupby(["L3 Workflow"])["Resolution"].mean().round(4)
          ],axis=1).head(10)

,Count,% Count,Resolution
User Disable Signin,2716,0.2973,0.1591
Unauthorized Crypto Buy/Sell/Send,731,0.0800,0.0559
Clawback Process,701,0.0767,0.1162
WBL Hold Time,515,0.0564,0.1560
Self Recovery Applied,325,0.0356,0.1733
Troubleshooting,293,0.0321,0.2136
CX Email Compromised,288,0.0315,0.1226
FAQ: The Password Reset Process,284,0.0311,0.2059
Unban Workflow,268,0.0293,0.0986
Unauthorized Password Reset,205,0.0224,0.1807


In [130]:
pd.concat([QUERY["Macro Applied"].value_counts().rename("Count"),
           QUERY["Macro Applied"].value_counts(normalize=True).round(4).rename("% Count"),
           QUERY.groupby(["Macro Applied"])["Resolution"].mean().round(4)
          ],axis=1).head(10)

,Count,% Count,Resolution
Self Recovery Already Processing,996,0.1092,0.1806
Resolved - Self Recovery,963,0.1056,0.1148
Password Reset,381,0.0418,0.0843
2FA - Resolved - Self Recovery,271,0.0297,0.0882
BETA T1 > Risk > Clawback - Money Owed,263,0.0288,0.1371
Password reset - Email in SPAM folder,252,0.0276,0.1092
WBL > Escalate to Decrease Cashout Availability Time Frame,248,0.0272,0.1143
Account Lockout - Balance Transfer Resolved,243,0.0266,0.2500
Temporary - Trust - Self Recovery Completed,237,0.0260,0.0929
AR Already Processing,214,0.0235,0.1500


In [68]:
DF_CONSOLIDATED["Case Number"].nunique()

242038

In [79]:
242038 / 1134431

0.2133562993253887

In [89]:
1094021 / 1134431

0.9643786180032105

#### Resolution Rate by # of Status Change

In [131]:
QUERY_RESO = \
pd.concat([QUERY.groupby(["Case Number","L1 Workflow"])["Status Change Prior"].count(),
           QUERY.groupby(["Case Number","L1 Workflow"])["Resolution"].mean()
          ],axis=1)

In [132]:
QUERY_RESO.groupby(["Status Change Prior"])["Resolution"].agg(["count","mean"])

,count,mean
Status Change Prior,,
1,3517,0.141882
2,65,0.061538
3,0,NaN


#### Resolution Rate by # of Days customer replied

In [125]:
QUERY_RESO = \
pd.concat([QUERY.groupby(["Case Number","L1 Workflow"])["Duration_Reopened"].value_counts().unstack().fillna(0).astype(int),
           QUERY.groupby(["Case Number","L1 Workflow"])["Resolution"].mean()
          ],axis=1)

In [155]:
pd.concat([QUERY_PHONE.groupby(["Duration_Reopened"])["Case Number"].nunique().rename("Count"),
           QUERY_PHONE["Duration_Reopened"].value_counts(normalize=True).round(4).rename("% Count"),
           QUERY_PHONE.groupby(["Duration_Reopened"])["Resolution"].mean().round(4).rename("Resolution %")
          ],
          axis=1
         ).reindex(["Same Day","1-2 Day","3-4 Days","5-7 Days", "> 7 Days"])

,Count,% Count,Resolution %
Same Day,12036,0.7745,0.1309
1-2 Day,2618,0.1192,0.0902
3-4 Days,1014,0.0435,0.1059
5-7 Days,752,0.0320,0.0891
> 7 Days,733,0.0308,0.1135


#### Phone: Deepdive - Same Day

In [196]:
QUERY = QUERY_PHONE[QUERY_PHONE["Duration_Reopened"] == "Same Day"]
QUERY = QUERY[~(QUERY["Case Number"].duplicated(keep="last"))].reset_index(drop=True)

In [197]:
pd.concat([QUERY["L1 Workflow"].value_counts().rename("Count"),
           QUERY["L1 Workflow"].value_counts(normalize=True).round(4).rename("% Count"),
           QUERY.groupby(["L1 Workflow"])["Resolution"].mean().round(4).rename("Resolution %")
          ],axis=1)

,Count,% Count,Resolution %
Identity Ops,5229,0.5646,0.1652
Ecosystem Ops,1287,0.1390,0.1723
Payment Ops,947,0.1023,0.2064
Retail Ops,527,0.0569,0.1550
Policy Ops,520,0.0561,0.2000
General,242,0.0261,0.1449
Institutional Ops,174,0.0188,0.3934
Banking Ops,142,0.0153,0.2667
Trading,140,0.0151,0.2581
Account Ops,51,0.0055,0.1818


In [185]:
pd.concat([QUERY["L2 Workflow"].value_counts().rename("Count"),
           QUERY["L2 Workflow"].value_counts(normalize=True).round(4).rename("% Count"),
           QUERY.groupby(["L2 Workflow"])["Resolution"].mean().round(4).rename("Resolution %")
          ],axis=1).head(10)

,Count,% Count,Resolution %
Account Safety,2464,0.2584,0.1401
Account Risk,1668,0.1749,0.1614
Account Access,1323,0.1387,0.2239
Crypto Ops,1283,0.1345,0.1723
ACH,659,0.0691,0.2181
Compliance Policy,347,0.0364,0.1792
Other,331,0.0347,0.2316
Wallet,272,0.0285,0.1102
API Support,209,0.0219,0.1429
Cards,179,0.0188,0.1231


In [186]:
pd.concat([QUERY["L3 Workflow"].value_counts().rename("Count"),
           QUERY["L3 Workflow"].value_counts(normalize=True).round(4).rename("% Count"),
           QUERY.groupby(["L3 Workflow"])["Resolution"].mean().round(4).rename("Resolution %")
          ],axis=1).head(10)

,Count,% Count,Resolution %
User Disable Signin,1442,0.1466,0.1753
Clawback Process,513,0.0522,0.1145
Unauthorized Crypto Buy/Sell/Send,439,0.0446,0.0437
Crypto Send Issues,328,0.0333,0.1810
WBL Hold Time,324,0.0329,0.1901
Crypto Receive Issues,240,0.0244,0.1205
Crypto TX Issues,237,0.0241,0.1515
Self Recovery Applied,234,0.0238,0.2283
Troubleshooting,215,0.0219,0.2667
FAQ: The Password Reset Process,211,0.0214,0.1803


In [187]:
pd.concat([QUERY["Macro Applied"].value_counts().rename("Count"),
           QUERY["Macro Applied"].value_counts(normalize=True).round(4).rename("% Count"),
           QUERY.groupby(["Macro Applied"])["Resolution"].mean().round(4).rename("Resolution %")
          ],axis=1).head(10)

,Count,% Count,Resolution %
Resolved - Self Recovery,625,0.0545,0.1360
Self Recovery Already Processing,528,0.0460,0.1591
General > Bug > Basic Troubleshooting - Clear Cache + Screenshot,425,0.0370,0.2337
Password Reset,282,0.0246,0.1081
Setup Trouble Resolved - Checking if OK,275,0.0240,0.3298
Temporary - Trust - Self Recovery Completed,242,0.0211,0.0833
Account Lockout - Balance Transfer Resolved,215,0.0187,0.2778
Fraud > Clawback - Account Compromise,211,0.0184,0.0550
Silent Resolve,204,0.0178,0.0652
BETA T1 > Risk > Clawback - Money Owed,203,0.0177,0.1163


In [222]:
SAMPLE = \
QUERY_PHONE.groupby(["Macro Applied"])["Duration_Reopened"].value_counts().unstack().fillna(0).astype(int)

SAMPLE["Total"] = SAMPLE.sum(axis=1)

#SAMPLE = SAMPLE.sort_values(by="Total", ascending=False).head(20)
#SAMPLE = SAMPLE[["Same Day","1-2 Day","3-4 Days","5-7 Days", "> 7 Days", "Total"]]

In [224]:
SAMPLE.div(SAMPLE.sum(axis=1),axis=0)

Duration_Reopened,1-2 Day,3-4 Days,5-7 Days,> 7 Days,Same Day
Macro Applied,,,,,
1099-K Amount and Calculations Explanation (1st Response),1.000000,0.000000,0.000000,0.000000,0.000000
1st Response - Hacked - Disabled Sign in,0.068966,0.068966,0.034483,0.034483,0.793103
1st Response - Lost/Stolen Device,0.000000,0.000000,0.000000,0.000000,1.000000
1st Response - Phone Support Scammed,0.000000,0.000000,0.000000,0.000000,1.000000
1st Response - User Disabled Signin,0.000000,0.000000,0.000000,0.000000,1.000000
...,...,...,...,...,...
de_DE - Product > How to Change Email Address,0.000000,0.000000,0.000000,0.000000,1.000000
de_DE - SEPA - Deposit Successful,1.000000,0.000000,0.000000,0.000000,0.000000
de_DE - SEPA - Missing Deposit,0.000000,0.000000,0.000000,0.000000,1.000000


## Chat Reopened Cases

In [109]:
QUERY_LM.groupby(["Workweek"])["Case Number"].nunique().rename("Count").reset_index()

,Workweek,Count
0,2021-12-06,118
1,2021-12-13,490
2,2021-12-20,1117
3,2021-12-27,1032


### Chat: Identity Ops

In [113]:
QUERY = QUERY_LM[QUERY_LM["L1 Workflow"] == "Identity Ops"].reset_index(drop=True)

In [114]:
pd.concat([QUERY["L2 Workflow"].value_counts().rename("Count"),
           QUERY["L2 Workflow"].value_counts(normalize=True).round(4).rename("% Count"),
           QUERY.groupby(["L2 Workflow"])["Resolution"].mean().round(4)
          ],axis=1)

,Count,% Count,Resolution
Account Risk,606,0.7922,0.1195
Account Access,76,0.0993,0.4286
Account Safety,61,0.0797,0.1304
Other,22,0.0288,0.3333


In [115]:
pd.concat([QUERY["L3 Workflow"].value_counts().rename("Count"),
           QUERY["L3 Workflow"].value_counts(normalize=True).round(4).rename("% Count"),
           QUERY.groupby(["L3 Workflow"])["Resolution"].mean().round(4)
          ],axis=1).head(10)

,Count,% Count,Resolution
WBL Hold Time,141,0.1843,0.0952
Unban Workflow,128,0.1673,0.0185
Clawback Process,120,0.1569,0.1842
Transfer Risk Model,65,0.0850,0.1071
Can't send crypto,54,0.0706,0.2308
Troubleshooting,46,0.0601,0.5882
Limit Increase Requests,29,0.0379,0.1250
Unauthorized Crypto Buy/Sell/Send,28,0.0366,0.2222
IDV Requirements For Escalation,23,0.0301,0.1250
TextMatch,19,0.0248,0.6000


In [116]:
pd.concat([QUERY["Macro Applied"].value_counts().rename("Count"),
           QUERY["Macro Applied"].value_counts(normalize=True).round(4).rename("% Count"),
           QUERY.groupby(["Macro Applied"])["Resolution"].mean().round(4)
          ],axis=1).head(10)

,Count,% Count,Resolution
WBL > Escalate to Decrease Cashout Availability Time Frame,70,0.0916,0.1034
Unban Rejected - Eligible for 30/60 Day Follow-Up,64,0.0838,0.0000
Fraud - Transfer Risk - Escalated,50,0.0654,0.1500
General > Bug > Basic Troubleshooting - Clear Cache + Screenshot,48,0.0628,0.6471
BETA T1 > Risk > Clawback - Money Owed,46,0.0602,0.1111
WBL > Cashout Availability Time Frame Sustained,39,0.0510,0.1667
Fraud > General - Pending Review,38,0.0497,0.0000
Clawback - Payment Pending,37,0.0484,0.2857
Risk > Disable Crypto Sends - Proactive Restriction,29,0.0380,0.2778
Setup Trouble Resolved - Checking if OK,14,0.0183,0.3750


#### Resolution Rate by # of Status Change

In [117]:
QUERY_RESO = \
pd.concat([QUERY.groupby(["Case Number","L1 Workflow"])["Status Change Prior"].count(),
           QUERY.groupby(["Case Number","L1 Workflow"])["Resolution"].mean()
          ],axis=1)

QUERY_RESO.groupby(["Status Change Prior"])["Resolution"].agg(["count","mean"])

,count,mean
Status Change Prior,,
1,273,0.161172
2,5,0.000000


#### Resolution Rate by # of Days customer replied

In [156]:
pd.concat([QUERY_LM.groupby(["Duration_Reopened"])["Case Number"].nunique().rename("Count"),
           QUERY_LM["Duration_Reopened"].value_counts(normalize=True).round(4).rename("% Count"),
           QUERY_LM.groupby(["Duration_Reopened"])["Resolution"].mean().round(4).rename("Resolution %")
          ],
          axis=1
         ).reindex(["Same Day","1-2 Day","3-4 Days","5-7 Days", "> 7 Days"])

,Count,% Count,Resolution %
Same Day,2383,0.8974,0.2594
1-2 Day,249,0.0641,0.0314
3-4 Days,78,0.0197,0.0222
5-7 Days,56,0.0135,0.0270
> 7 Days,22,0.0052,0.0833


#### Chat: Deepdive - Same Day

In [188]:
QUERY = QUERY_LM[QUERY_LM["Duration_Reopened"] == "Same Day"]
QUERY = QUERY[~(QUERY["Case Number"].duplicated(keep="last"))].reset_index(drop=True)

In [190]:
pd.concat([QUERY["L1 Workflow"].value_counts().rename("Count"),
           QUERY["L1 Workflow"].value_counts(normalize=True).round(4).rename("% Count"),
           QUERY.groupby(["L1 Workflow"])["Resolution"].mean().round(4).rename("Resolution %")
          ],axis=1)

,Count,% Count,Resolution %
Identity Ops,545,0.2690,0.2111
Ecosystem Ops,402,0.1984,0.3509
Policy Ops,330,0.1629,0.3448
Institutional Ops,308,0.1520,0.5053
Retail Ops,221,0.1091,0.2553
Payment Ops,191,0.0943,0.2273
Trading,15,0.0074,0.0000
Banking Ops,11,0.0054,0.0000
General,3,0.0015,0.5000


In [192]:
pd.concat([QUERY["L2 Workflow"].value_counts().rename("Count"),
           QUERY["L2 Workflow"].value_counts(normalize=True).round(4).rename("% Count"),
           QUERY.groupby(["L2 Workflow"])["Resolution"].mean().round(4).rename("Resolution %")
          ],axis=1).head(10)

,Count,% Count,Resolution %
Account Risk,429,0.2100,0.1748
Crypto Ops,391,0.1914,0.3604
Pro,288,0.1410,0.5393
Retail Onboarding,163,0.0798,0.4222
Compliance Policy,162,0.0793,0.2750
ACH,133,0.0651,0.2727
Wallet,120,0.0587,0.2381
Other,66,0.0323,0.1875
Account Access,63,0.0308,0.5714
Account Safety,47,0.0230,0.0588


In [193]:
pd.concat([QUERY["L3 Workflow"].value_counts().rename("Count"),
           QUERY["L3 Workflow"].value_counts(normalize=True).round(4).rename("% Count"),
           QUERY.groupby(["L3 Workflow"])["Resolution"].mean().round(4).rename("Resolution %")
          ],axis=1).head(10)

,Count,% Count,Resolution %
IDV (Jumio),104,0.0501,0.4667
Crypto Send Issues,97,0.0468,0.6000
WBL Hold Time,95,0.0458,0.1111
Clawback Process,90,0.0434,0.2857
Pro Mobile App,88,0.0424,0.5789
Unban Workflow,84,0.0405,0.0333
Wallet Send Issues,68,0.0328,0.2667
Deposits and Withdrawals (Crypto),61,0.0294,0.5294
Wallet Quick Wins,57,0.0275,0.2000
Crypto Receive Issues,55,0.0265,0.3333


In [194]:
pd.concat([QUERY["Macro Applied"].value_counts().rename("Count"),
           QUERY["Macro Applied"].value_counts(normalize=True).round(4).rename("% Count"),
           QUERY.groupby(["Macro Applied"])["Resolution"].mean().round(4).rename("Resolution %")
          ],axis=1).head(10)

,Count,% Count,Resolution %
General > Bug > Basic Troubleshooting - Clear Cache + Screenshot,230,0.0986,0.4583
T1 Pro General > Free Form Template #1 - General,72,0.0309,0.6400
Setup Trouble Resolved - Checking if OK,65,0.0279,0.4000
Unban Rejected - Eligible for 30/60 Day Follow-Up,50,0.0214,0.0000
WBL > Escalate to Decrease Cashout Availability Time Frame,47,0.0202,0.1176
Fraud - Transfer Risk - Escalated,46,0.0197,0.2222
Crypto Successfully Sent Out of CB,43,0.0184,0.5333
Pages - Balance and Transactions,40,0.0172,0.4545
WBL > Cashout Availability Time Frame Sustained,39,0.0167,0.1111
BETA T1 > Risk > Clawback - Money Owed,38,0.0163,0.1250


In [70]:
QUERY_WEB = QUERY_WEB[QUERY_WEB["L1 Workflow"] == "Identity Ops"]

In [85]:
pd.concat([QUERY_WEB.groupby(["Macro Applied"])["Case Number"].nunique(),
           QUERY_WEB.groupby(["Macro Applied"])["Resolution"].mean()
          ],axis=1).head(10).sort_values(by="Case Number",ascending=False)

,Case Number,Resolution
Macro Applied,,
2FA - Phone Change (SMS),1427,0.215170
2FA - Email - Password Reset Link Error,273,0.277027
2FA - Email Verification Fail,228,0.203540
2FA - After deleting a zero balance account,129,0.652174
2FA - Phone Change From Within The Account,47,0.285714
1st Response - User Disabled Signin,18,0.222222
1st Response - Hacked - Disabled Sign in,8,0.000000
1st Response - Lost/Stolen Device,3,0.500000
10/27/21 Incident,1,NaN


In [58]:
QUERY = \
DF_CONSOLIDATED.groupby(["L1 Workflow", "Case Origin"])["Case Number"].nunique().unstack().fillna(0).astype(int)

QUERY["Total_Count"] = QUERY.sum(axis=1)
QUERY = QUERY.sort_values(by="Total_Count", ascending=False)
QUERY["Total_Count %"] = QUERY["Total_Count"].div(QUERY["Total_Count"].sum())

In [59]:
QUERY

Case Origin,Live Message,Phone,Web,Total_Count,Total_Count %
L1 Workflow,,,,,
Identity Ops,735,6476,54450,61661,0.323768
Ecosystem Ops,499,1555,35452,37506,0.196936
Policy Ops,412,636,26071,27119,0.142396
Payment Ops,263,1043,24977,26283,0.138006
Retail Ops,291,637,23113,24041,0.126234
Institutional Ops,400,212,7821,8433,0.044280
Trading,18,213,2775,3006,0.015784
Banking Ops,18,161,1438,1617,0.008491
General,5,269,501,775,0.004069


In [62]:
QUERY_PHONE = \
DF_CONSOLIDATED[DF_CONSOLIDATED["Case Origin"] == "Phone"].reset_index(drop=True)

In [63]:
QUERY_PHONE

,Case Number,Status Change Prior,Date/Time Reopened,L1 Workflow,L2 Workflow,L3 Workflow,Macro Applied,Interval (Days),Days_Check,Workweek,Case Origin
0,7849259,2021-10-13 09:28:00,2021-10-13 09:53:00,Identity Ops,Account Safety,User Disable Signin,Account Recovery - SMS Not Received,0,Within 5 Days,2021-10-11,Phone
1,7849032,2021-10-13 14:28:00,2021-10-13 17:59:00,Identity Ops,Account Safety,User Disable Signin,Self Recovery Already Processing,0,Within 5 Days,2021-10-11,Phone
2,7849032,2021-10-19 17:49:00,2021-10-28 19:01:00,NaN,NaN,NaN,Resolved - Restoring w/ TOTP Removal,9,Outside 5 Days,2021-10-25,Phone
3,7847663,2021-10-13 08:53:00,2021-10-13 09:02:00,Identity Ops,Account Safety,User Disable Signin,Jumio - Manually Verified,0,Within 5 Days,2021-10-11,Phone
4,7847663,2021-10-13 11:03:00,2021-10-13 11:56:00,NaN,NaN,NaN,CB Declined Transactions Reasons,0,Within 5 Days,2021-10-11,Phone
...,...,...,...,...,...,...,...,...,...,...,...
17609,9259942,2021-12-27 03:05:00,2021-12-28 22:03:00,NaN,NaN,NaN,Silent Resolve,1,Within 5 Days,2021-12-27,Phone
17610,9259942,2021-12-29 16:43:00,2021-12-31 09:20:00,NaN,NaN,NaN,2FA - Phone Change (SMS),1,Within 5 Days,2021-12-27,Phone
17611,9260359,2021-12-29 20:30:00,2021-12-30 15:13:00,Identity Ops,Account Safety,User Disable Signin,Password Reset,0,Within 5 Days,2021-12-27,Phone
17612,9260400,2021-12-27 01:54:00,2021-12-27 02:19:00,Policy Ops,Compliance Policy,FCA,FCA High Risk - Final Response,0,Within 5 Days,2021-12-27,Phone


In [257]:
QUERY

,Case Number,Status Change Prior,Date/Time Reopened,L1 Workflow,L2 Workflow,L3 Workflow,Macro Applied,Interval (Days),Days_Check,Workweek,Resolution
0,7669192,2021-10-01 06:35:00,2021-10-01 06:40:00,NaN,NaN,NaN,General > Free Form Template #2 - Empathetic,0,Within 5 Days,2021-09-27,0.0
1,7683964,2021-10-02 17:04:00,2021-10-02 18:31:00,NaN,NaN,NaN,NaN,0,Within 5 Days,2021-09-27,0.0
2,7683964,2021-10-02 16:28:00,2021-10-02 16:49:00,NaN,NaN,NaN,General > Free Form Template #1 - General,0,Within 5 Days,2021-09-27,0.0
3,7683964,2021-10-02 14:29:00,2021-10-02 14:33:00,NaN,NaN,NaN,Known Issue - Closing,0,Within 5 Days,2021-09-27,0.0
4,7634520,2021-10-02 11:29:00,2021-10-02 14:17:00,NaN,PayPal,Link Paypal Account,Setup Trouble Resolved - Checking if OK,0,Within 5 Days,2021-09-27,NaN
...,...,...,...,...,...,...,...,...,...,...,...
444009,9260119,2021-12-27 00:26:00,2021-12-27 00:43:00,Policy Ops,Compliance Policy,Account Closure (MAC),MAC - No Terms,0,Within 5 Days,2021-12-27,NaN
444010,9260108,2021-12-27 01:05:00,2021-12-27 01:07:00,Ecosystem Ops,Commerce,Commerce Merchant Account Access,Commerce> Account Issues- Lost Seed Phrase,0,Within 5 Days,2021-12-27,0.0
444011,9260103,2021-12-27 06:40:00,2021-12-27 15:35:00,NaN,NaN,NaN,Coinbase Wallet - Reset Transfer Limit,0,Within 5 Days,2021-12-27,NaN
444012,9260155,2021-12-27 23:03:00,2021-12-27 23:37:00,NaN,NaN,NaN,Coinbase Wallet - Adjusting Miner Fees 101,0,Within 5 Days,2021-12-27,NaN


## Identity Ops

In [305]:
QUERY = DF_CONSOLIDATED[DF_CONSOLIDATED["L1 Workflow"] == "Identity Ops"]

In [306]:
QUERY = QUERY.merge(DF_SURVEY[["Case Number", "Resolution"]], on="Case Number", how="left")

In [307]:
QUERY["Resolution"].isnull().apply(lambda x: 1 if x==True else 0).mean()

0.5010598091222089

In [308]:
QUERY.head()

,Case Number,Status Change Prior,Date/Time Reopened,L1 Workflow,L2 Workflow,L3 Workflow,Macro Applied,Interval (Days),Days_Check,Workweek,Resolution
0,7636165,2021-10-01 00:08:00,2021-10-01 03:34:00,Identity Ops,Account Safety,User Disable Signin,IDV - Troubleshooting - Mobile App and Camera,0,Within 5 Days,2021-09-27,NaN
1,7640101,2021-10-01 12:30:00,2021-10-01 12:40:00,Identity Ops,Other,NaN,Setup Trouble Resolved - Checking if OK,0,Within 5 Days,2021-09-27,NaN
2,7639499,2021-10-01 07:30:00,2021-10-01 07:31:00,Identity Ops,Account Safety,CX Email Compromised,Silent Resolve,0,Within 5 Days,2021-09-27,NaN
3,7641557,2021-10-01 10:23:00,2021-10-01 12:26:00,Identity Ops,Account Access,Self Recovery Applied,Multiple Accounts - Mix Up,0,Within 5 Days,2021-09-27,0.0
4,7617547,2021-10-01 05:16:00,2021-10-03 08:43:00,Identity Ops,Account Access,Troubleshooting,Setup Trouble Resolved - Checking if OK,2,Within 5 Days,2021-09-27,NaN


In [309]:
QUERY.groupby(["Workweek", "Days_Check"])["Case Number"].nunique().unstack()

Days_Check,Outside 5 Days,Within 5 Days
Workweek,,
2021-09-27,NaN,129.0
2021-10-04,2.0,4997.0
2021-10-11,115.0,6695.0
2021-10-18,146.0,6625.0
2021-10-25,184.0,7480.0
2021-11-01,167.0,9359.0
2021-11-08,238.0,8631.0
2021-11-15,222.0,7957.0
2021-11-22,166.0,6800.0


In [310]:
QUERY.groupby(["Case Number","L1 Workflow"])["Status Change Prior"].count().unstack().mean()

L1 Workflow
Identity Ops    1.019704
dtype: float64

In [311]:
QUERY.groupby(["L2 Workflow"])["Resolution"].mean()

L2 Workflow
ACH                       NaN
Account Access       0.204614
Account Risk         0.149774
Account Safety       0.132235
Coinbase One              NaN
Commerce             0.000000
Compliance Policy    0.250000
Crypto Ops           0.000000
Facematch                 NaN
Other                0.187361
Pro                       NaN
Push Payments             NaN
Retail Onboarding    0.000000
Trading Ops               NaN
Name: Resolution, dtype: float64

In [312]:
pd.concat([QUERY["L2 Workflow"].value_counts().rename("Count"),
           QUERY["L2 Workflow"].value_counts(normalize=True).rename("Count"),
           QUERY.groupby(["L2 Workflow"])["Resolution"].mean()
          ],axis=1)

,Count,Count,Resolution
Account Risk,41360,0.465116,0.149774
Account Access,24228,0.272457,0.204614
Account Safety,16892,0.189960,0.132235
Other,6401,0.071983,0.187361
Compliance Policy,24,0.000270,0.250000
Retail Onboarding,10,0.000112,0.000000
Crypto Ops,2,0.000022,0.000000
Commerce,1,0.000011,0.000000
Pro,1,0.000011,NaN
Facematch,1,0.000011,NaN


In [313]:
pd.concat([QUERY["L3 Workflow"].value_counts().rename("Count"),
           QUERY["L3 Workflow"].value_counts(normalize=True).rename("Count"),
           QUERY.groupby(["L3 Workflow"])["Resolution"].mean()
          ],axis=1).head(20)

,Count,Count,Resolution
WBL Hold Time,7733,0.087189,0.167610
Clawback Process,7666,0.086434,0.148466
Unban Workflow,6338,0.071461,0.148893
User Disable Signin,6015,0.067819,0.157688
Troubleshooting,4415,0.049779,0.195225
Customer Loses Access to 2FA - SMS,4270,0.048144,0.225955
Transfer Risk Model,4056,0.045731,0.068199
TextMatch,4051,0.045675,0.139417
Press Workflow,3877,0.043713,0.187965
Unauthorized Crypto Buy/Sell/Send,3302,0.037230,0.085714


In [314]:
pd.concat([QUERY["Macro Applied"].value_counts().rename("Count"),
           QUERY["Macro Applied"].value_counts(normalize=True).rename("Count"),
           QUERY.groupby(["Macro Applied"])["Resolution"].mean()
          ],axis=1).head(20)

,Count,Count,Resolution
Resolved - Self Recovery,4144,0.046597,0.112754
BETA T1 > Risk > Clawback - Money Owed,3781,0.042515,0.122500
WBL > Escalate to Decrease Cashout Availability Time Frame,3624,0.040750,0.086507
Fraud - Transfer Risk - Escalated,3427,0.038535,0.064459
General > Bug > Basic Troubleshooting - Clear Cache + Screenshot,3009,0.033834,0.205761
WBL > Cashout Availability Time Frame Sustained,3000,0.033733,0.277778
IDV - Textmatch,2986,0.033576,0.145952
General - Email Not Registered,2871,0.032283,0.150538
General > Free Form Template #1 - General,2631,0.029584,0.140490
2FA - SMS Fail,2131,0.023962,0.194076


In [318]:
QUERY[QUERY["L3 Workflow"] == "WBL Hold Time"]["Macro Applied"].value_counts().head(10)

WBL > Escalate to Decrease Cashout Availability Time Frame    3578
WBL > Cashout Availability Time Frame Sustained               2962
WBL > Cashout Availability General                             250
General > Free Form Template #1 - General                      138
Fraud > General - Pending Review                               127
WBL > Cashout Availability Transfer to Pro                      63
Setup Trouble Resolved - Checking if OK                         52
General > Merged Case                                           50
General - Issue Resolved                                        47
Silent Resolve                                                  36
Name: Macro Applied, dtype: int64

## Ecosystem

In [245]:
QUERY = DF_CONSOLIDATED[DF_CONSOLIDATED["L1 Workflow"] == "Ecosystem Ops"]

In [246]:
QUERY = QUERY.merge(DF_SURVEY[["Case Number", "Resolution"]], on="Case Number", how="left")

In [249]:
QUERY["Resolution"].isnull().apply(lambda x: 1 if x==True else 0).mean()

0.48280628668812725

In [241]:
QUERY.groupby(["Workweek", "Days_Check"])["Case Number"].nunique().unstack()

Days_Check,Outside 5 Days,Within 5 Days
Workweek,,
2021-09-27,NaN,28.0
2021-10-04,2.0,3416.0
2021-10-11,27.0,4225.0
2021-10-18,24.0,4158.0
2021-10-25,24.0,3741.0
2021-11-01,33.0,5414.0
2021-11-08,20.0,5089.0
2021-11-15,13.0,5000.0
2021-11-22,27.0,4945.0


In [189]:
QUERY.groupby(["Case Number","L1 Workflow"])["Status Change Prior"].count().unstack().mean()

L1 Workflow
Ecosystem Ops    1.030093
dtype: float64

In [242]:
pd.concat([QUERY["L2 Workflow"].value_counts().rename("Count"),
           QUERY["L2 Workflow"].value_counts(normalize=True).rename("Count"),
           QUERY.groupby(["L2 Workflow"])["Resolution"].mean()
          ],axis=1)

,Count,Count,Resolution
Crypto Ops,51654,0.979223,0.151743
Commerce,939,0.017801,0.148148
Scaled Developer Support,85,0.001611,0.068182
NFT - Support,48,0.000910,0.000000
Other,5,0.000095,0.000000
Account Risk,4,0.000076,0.000000
Compliance Policy,4,0.000076,NaN
Account Access,3,0.000057,NaN
Account Safety,3,0.000057,NaN
Wallet,2,0.000038,NaN


In [243]:
pd.concat([QUERY["L3 Workflow"].value_counts().rename("Count"),
           QUERY["L3 Workflow"].value_counts(normalize=True).rename("Count"),
           QUERY.groupby(["L3 Workflow"])["Resolution"].mean()
          ],axis=1).head(20)

,Count,Count,Resolution
Crypto Send Issues,8223,0.156459,0.238848
Crypto Receive Issues,7823,0.148848,0.109554
Crypto Quick Wins,7791,0.148239,0.120662
Crypto TX Issues,5890,0.112069,0.102431
Wallet Conversions,5356,0.101908,0.079487
Wallet Send Issues,4793,0.091196,0.128856
Wallet Receive Issues,4425,0.084194,0.208732
Wallet,3223,0.061324,0.130551
Crypto Question Flow,2259,0.042982,0.133929
XTZ Staking Issues,722,0.013737,0.133858


In [244]:
pd.concat([QUERY["Macro Applied"].value_counts().rename("Count"),
           QUERY["Macro Applied"].value_counts(normalize=True).rename("Count"),
           QUERY.groupby(["Macro Applied"])["Resolution"].mean()
          ],axis=1).head(20)

,Count,Count,Resolution
Crypto Successfully Sent Out of CB,3623,0.069187,0.274490
General > Free Form Template #1 - General,2723,0.052000,0.136868
Crypto - BEP2,2218,0.042357,0.073338
General > Bug > Basic Troubleshooting - Clear Cache + Screenshot,2115,0.040390,0.123277
Coinbase Wallet - Bug Troubleshooting,1617,0.030879,0.107252
Pages - Balance and Transactions,1586,0.030287,0.141304
Crypto - Unsupported Network,1385,0.026449,0.054945
Coinbase Wallet - Miner Fee Error,1265,0.024157,0.102686
Coinbase Wallet - Funds Sent via Unsupported Network,1139,0.021751,0.143599
Crypto - Tx Hash Explain,1069,0.020414,0.171717


## Status

### All Resolved Status Movement

In [63]:
QUERY = \
DF_HISTORY_STATUS[DF_HISTORY_STATUS["Old Value"] == "Resolved"]

In [71]:
pd.concat([QUERY["New Value"].value_counts().rename("Count"),
           (QUERY["New Value"].value_counts(normalize=True).rename("% Count") * 100).round(2)
          ],axis=1)

,Count,% Count
Closed,708525,57.72
Customer Responded,449380,36.61
Open - In Progress,26686,2.17
Pending Customer,21286,1.73
Escalated,8734,0.71
Transferred,7252,0.59
Pending Internal,3893,0.32
New,1742,0.14


In [68]:
DF_HISTORY_STATUS["New Value"].value_counts()

Open - In Progress      2551829
Resolved                1409978
Closed                  1137806
Customer Responded       886865
Pending Customer         515002
Transferred              345099
Escalated                201036
Pending Internal         139775
Ready for Agent           92158
New                       21873
Assigned to Reviewer         33
Name: New Value, dtype: int64

In [46]:
QUERY = \
DF_HISTORY_STATUS[(DF_HISTORY_STATUS["Old Value"] == "Resolved") & (DF_HISTORY_STATUS["New Value"] == "Customer Responded")]

In [53]:
QUERY.groupby(DF_HISTORY_STATUS["Edit Week"])["Case Number"].nunique().reset_index()

,Edit Week,Case Number
0,2021-09-27,5654
1,2021-10-04,19556
2,2021-10-11,19998
3,2021-10-18,20767
4,2021-10-25,23902
5,2021-11-01,31491
6,2021-11-08,29901
7,2021-11-15,27663
8,2021-11-22,24700
9,2021-11-29,22840


In [48]:
DF_HISTORY_STATUS[DF_HISTORY_STATUS["Old Value"] == "Resolved"]["New Value"].value_counts()

Closed                708525
Customer Responded    449380
Open - In Progress     26686
Pending Customer       21286
Escalated               8734
Transferred             7252
Pending Internal        3893
New                     1742
Name: New Value, dtype: int64

In [49]:
DF_HISTORY_STATUS["Old Value"] == "Resolved"

0          False
1          False
2          False
3           True
4          False
           ...  
7301453    False
7301454    False
7301455    False
7301456    False
7301457    False
Name: Old Value, Length: 7301458, dtype: bool